In [ ]:
# !pip install grad-cam
!pip install ttach
!pip install timm

     |████████████████████████████████| 376 kB 35.4 MB/s 


In [ ]:
!git clone https://github.com/vigneshuw/pytorch-grad-cam.git
!pip install pytorch-grad-cam/

Cloning into 'pytorch-grad-cam'...
remote: Enumerating objects: 702, done.
remote: Counting objects: 100% (241/241), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 702 (delta 139), reused 160 (delta 82), pack-reused 461
Receiving objects: 100% (702/702), 1.90 MiB | 32.90 MiB/s, done.
Resolving deltas: 100% (367/367), done.
Processing ./pytorch-grad-cam
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 60.3 MB 1.3 MB/s 
  Created wheel for grad-cam: filename=gr

In [ ]:
# Google Collab Requirements / Checks
from google.colab.patches import cv2_imshow # Change this before running elsewhere!!

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# cd into yolo directory
%cd "/content/gdrive/MyDrive/University of Wisconsin-Madison/2021 Fall/cs762/CS762_Deep_Learning_Project/yolov3"
# %cd "/content/gdrive/MyDrive/CS762_Deep_Learning_Project/yolov3"

/content/gdrive/.shortcut-targets-by-id/137Ds-GMPEANJbJ-EgO6RczyM3GlmpY1n/CS762_Deep_Learning_Project/yolov3


In [ ]:
# Imports
import os
import cv2
import glob
import gc
import numpy as np
import pandas as pd
import copy
import math

from PIL import Image

import torch
from torch.autograd import Variable
from torchvision import transforms

import darknet as dn
from darknet_util import write_results

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image

In [ ]:
# Load manual ms coco dataset and bounding boxes
# Varun's Directory
dir_uri = '/content/gdrive/MyDrive/University of Wisconsin-Madison/2021 Fall/cs762'
# Devesh's Directory
# dir_uri = '/content/gdrive/MyDrive'

proj_dir = dir_uri + '/CS762_Deep_Learning_Project'
# df = pd.read_excel( proj_dir + '/fullBBOX.xlsx')

dataset_folder = proj_dir + '/Q7/new_dataset' 

image_names = [
          'person_backpack.jpg', 'person-no_backpack.jpg',
          'person_car.jpeg', 'person-no_car.jpeg'
          # 'dining table_chair.jpeg', 'dining table-no_chair.jpeg'
]

# Yolo COCO classes (80 only)
def load_classes(namesfile):
    fp = open(namesfile, "r")
    names = fp.read().split("\n")[:-1]
    return names

yolo_classes = load_classes("data/coco.names")

# DETR COCO classes (91 - acutally 80 when N/A removed)
DETR_CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# Load iamge net categories
imageNet_labels_path = dir_uri + "/CS762_Deep_Learning_Project/datasets/imagenet_classes.txt"
# Read the categories
with open(imageNet_labels_path, "r") as f:
    categories = [s.strip() for s in f.readlines()]

# Load bbox's
# bbox_path = proj_dir + '/fullBBOX.xlsx'
# bbox_df = pd.read_excel(bbox_path)
# all_boxes = bbox_df.to_numpy()

output_file = proj_dir + '/Q7/q7_trial2.csv'
headers = ['image_name', 'threshold', 'yolo label', 'yolo accuracy', 'yolo whole image %', 'detr label', 'detr accuracy', 'detr whole image %']

In [ ]:
yolo_transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor(),
])

detr_transform = transforms.Compose([
    transforms.Resize(800),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


# bbox given as one row from our fullBBox.xlsx file
def eval_metric_percent_above(grayscale_cam, bbox, threshold):

    # find area of bbox
    topLeft_w = int(bbox[2])
    topLeft_h = int(bbox[3])
    botRight_w = int(bbox[6])
    botRight_h = int(bbox[7])
    width_bbox = abs(botRight_w - topLeft_w)
    height_bbox = abs(topLeft_h - botRight_h)
    area = width_bbox * height_bbox

    # percent above threshold whole image
    above_threshold = (grayscale_cam > threshold)
    count_above_threshold = above_threshold.sum()
    total_count = grayscale_cam.shape[0] * grayscale_cam.shape[1]
    percent_above_threshold_whole_image = (count_above_threshold / total_count) * 100

    # percent above threshold in bbox
    bbox_grayscale_cam = grayscale_cam[topLeft_h:botRight_h, topLeft_w:botRight_w]
    bbox_above_threshold = (bbox_grayscale_cam > threshold)
    bbox_count_above_threshold = bbox_above_threshold.sum()
    bbox_total_count = bbox_grayscale_cam.shape[0] * bbox_grayscale_cam.shape[1]
    percent_above_threshold_in_bbox = (bbox_count_above_threshold / bbox_total_count) * 100

    # percent above threshold outside bbox
    outside_bbox_cam = copy.deepcopy(grayscale_cam)
    outside_bbox_cam[topLeft_h:botRight_h, topLeft_w:botRight_w] = np.zeros((height_bbox, width_bbox))
    outside_bbox_above_threshold = (outside_bbox_cam > threshold)
    outside_bbox_count_above_threshold = outside_bbox_above_threshold.sum()
    outside_bbox_total_count = total_count - bbox_total_count
    percent_above_threshold_outside_bbox = (outside_bbox_count_above_threshold / outside_bbox_total_count) * 100

    # return
    return percent_above_threshold_whole_image, percent_above_threshold_in_bbox, percent_above_threshold_outside_bbox

def eval_metric_percent_above_simple(grayscale_cam, threshold):
    # percent above threshold whole image
    above_threshold = (grayscale_cam > threshold)
    count_above_threshold = above_threshold.sum()
    total_count = grayscale_cam.shape[0] * grayscale_cam.shape[1]
    percent_above_threshold_whole_image = (count_above_threshold / total_count) * 100

    # return
    return percent_above_threshold_whole_image

def resizeBbox(orig_bbox, orig_img_size, new_img_size):
    orig_h = orig_img_size[0]
    orig_w = orig_img_size[1]
    new_h = new_img_size[0]
    new_w = new_img_size[1]

    h_ratio = new_h / orig_h
    w_ratio = new_w / orig_w

    new_bbox = copy.deepcopy(orig_bbox)
    new_bbox[2] = orig_bbox[2] * w_ratio
    new_bbox[3] = orig_bbox[3] * h_ratio
    new_bbox[4] = orig_bbox[4] * w_ratio
    new_bbox[5] = orig_bbox[5] * h_ratio
    new_bbox[6] = orig_bbox[6] * w_ratio
    new_bbox[7] = orig_bbox[7] * h_ratio
    new_bbox[8] = orig_bbox[8] * w_ratio
    new_bbox[9] = orig_bbox[9] * h_ratio

    return new_bbox

In [ ]:
yolo_transform_eval = transforms.Compose([
    transforms.Resize((416, 416)),
])

detr_transform_eval = transforms.Compose([
    transforms.Resize(800),
])


def show_cam_on_image_updated(img: np.ndarray,
                      mask: np.ndarray,
                      use_rgb: bool = False,
                      colormap: int = cv2.COLORMAP_JET) -> np.ndarray:
    """ This function overlays the cam mask on the image as an heatmap.
    By default the heatmap is in BGR format.
    :param img: The base image in RGB or BGR format.
    :param mask: The cam mask.
    :param use_rgb: Whether to use an RGB or BGR heatmap, this should be set to True if 'img' is in RGB format.
    :param colormap: The OpenCV colormap to be used.
    :returns: The default image with the cam overlay.
    """
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), colormap)
    if use_rgb:
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    heatmap = np.float32(heatmap) / 255

    if np.max(img) > 1:
        raise Exception(
            "The input image should np.float32 in the range [0, 1]")

    cam = heatmap + img
    cam = cam / np.max(cam)
    return heatmap, np.uint8(255 * cam)


def visualize_eval_metric(rgb_image, grayscale_cam, heatmap, bbox, threshold):
    red_color = (0, 0, 255)
    thickness = 2
    start_point = (int(bbox[2]), int(bbox[3]))
    end_point = (int(bbox[6]), int(bbox[7]))

    above_threshold = (grayscale_cam > threshold)

    for i in range(grayscale_cam.shape[0]):
      for j in range(grayscale_cam.shape[1]):
        if(above_threshold[i, j] == False):
          heatmap[i,j,0] = 0
          heatmap[i,j,1] = 0
          heatmap[i,j,2] = 0

    cam = heatmap + rgb_image
    cam = cam / np.max(cam)
    cam = np.uint8(255 * cam)
    cam = cv2.rectangle(cam, start_point, end_point, red_color, thickness)
    return cam

# for output bounding box post-processing
def detr_box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def detr_rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = detr_box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

# PLotting the results
def get_detr_label_and_prob(pil_img, prob, boxes, target_class):
    label = None
    max_prob = 0

    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        cl = p.argmax()
        if (DETR_CLASSES[cl] == target_class):
          if p[cl] > max_prob:
            label = DETR_CLASSES[cl]
            max_prob = p[cl]
    
    return label, max_prob

In [ ]:
torch.set_grad_enabled(True)

blocks = dn.parse_cfg("cfg/yolov3.cfg")
yolo_model = dn.create_modules(blocks)

yolo_model = dn.Darknet("cfg/yolov3.cfg", False)

yolo_model.load_weights("weights/yolov3.weights")
yolo_model.eval();

#### Adding the gradient function to the tensors
for param in yolo_model.parameters():
    param.requires_grad = True

detr_model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
detr_model.eval();

 #### Adding the gradient function to the tensors
for param in detr_model.parameters():
    param.requires_grad = True

Downloading: "https://github.com/facebookresearch/detr/archive/main.zip" to /root/.cache/torch/hub/main.zip
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Downloading: "https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth" to /root/.cache/torch/hub/checkpoints/detr-r50-e632da11.pth


  0%|          | 0.00/159M [00:00<?, ?B/s]

In [ ]:
yolo_target_layers = [yolo_model.module_list[77].conv_77]
detr_target_layers = [detr_model.input_proj]

target_category = None

yolo_input_image_shape = (416, 416, 3)
#detr_input_image_shape = (800, ___, 3)

threshold_options = [0.7, 0.9]

In [ ]:
output_data = []

for threshold in threshold_options:
  for i in range(len( image_names ) ):
    print(image_names[i])

    # Create input
    test_img = Image.open( dataset_folder + '/' + image_names[i] ).convert('RGB')
    # orig_bbox = all_boxes[i]
    # orig_img_shape = np.array(test_img).shape

    # Create target lable
    target_name = image_names[i].split( '-' )[0]
    target_name = target_name.split( '_' )[0]

    #
    # YOLO
    #
    
    # YOLO Target label
    # handle mismatch spelling in yolo classes
    yolo_target_name = target_name
    if yolo_target_name == 'airplane':
      yolo_target_name = 'aeroplane'
    if yolo_target_name == 'dining table':
      yolo_target_name = 'diningtable'
      
    target_category = [index for index, class_instance in enumerate(yolo_classes) if class_instance == yolo_target_name]

    # Input
    input_tensor = yolo_transform(test_img)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # Get output
    with torch.no_grad():
      output = yolo_model(input_batch)

    yolo_predicted_label = ''
    yolo_predicted_label_prob = 0

    output = write_results(output[0], 0.5, 80)
    # Find box with target class
    if type(output) != int:
      for box in output:
        if target_category[0] == int(box[7]):
          yolo_predicted_label_prob = 1 / (1 + math.exp(-float(box[6])))
          yolo_predicted_label = target_name
          break

    if ( yolo_predicted_label == '' ):
      print( "YOLO did not get true classification for: {}".format( image_names[i] ) )

    # Grad cam time
    yolo_cam = GradCAM(model=yolo_model, target_layers=yolo_target_layers, use_cuda=False, reshape_transform=None, classification_logit="pred_logits")
    yolo_grayscale_cam = yolo_cam(input_tensor=input_batch, target_category=target_category, eigen_smooth=False, aug_smooth=False)
    yolo_grayscale_cam = yolo_grayscale_cam[0, :]

    # resize bbox and calculate metrics
    # new_bbox_yolo = resizeBbox(orig_bbox, (orig_img_shape[0], orig_img_shape[1]), (yolo_input_image_shape[0], yolo_input_image_shape[1]))
    # yolo_whole_img, yolo_in_bbox, yolo_out_bbox = eval_metric_percent_above(yolo_grayscale_cam, new_bbox_yolo, threshold)
    yolo_whole_img = eval_metric_percent_above_simple( yolo_grayscale_cam, threshold )


    #
    # DETR
    #

    # DETR Target Label
    target_category = [index for index, class_instance in enumerate(DETR_CLASSES) if class_instance == target_name]

    # Input
    input_tensor = detr_transform(test_img)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # Compute the predictions
    with torch.no_grad():
      output = detr_model(input_batch)

    # and convert them into probabilities
    # keep only predictions with 0.7+ confidence
    probas = output['pred_logits'].softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.9

    # finally get the index of the prediction with highest score
    detr_predicted_label = None
    detr_predicted_label_prob = 0

    # convert boxes from [0; 1] to image scales
    bboxes_scaled = detr_rescale_bboxes(output['pred_boxes'][0, keep], test_img.size)

    detr_predicted_label, detr_predicted_label_prob = get_detr_label_and_prob(test_img, probas[keep], bboxes_scaled, target_name)

    # Grad cam time
    detr_cam = GradCAM(model=detr_model, target_layers=detr_target_layers, use_cuda=False, reshape_transform=None, classification_logit="pred_logits")
    detr_grayscale_cam = detr_cam(input_tensor=input_batch, target_category=target_category, eigen_smooth=False, aug_smooth=False)
    detr_grayscale_cam = detr_grayscale_cam[0, :]

    # resize bbox and calculate metrics
    # detr_input_image_shape = (detr_grayscale_cam.shape[0], detr_grayscale_cam.shape[1], 3)
    # new_bbox_detr = resizeBbox(orig_bbox, (orig_img_shape[0], orig_img_shape[1]), (detr_input_image_shape[0], detr_input_image_shape[1]))
    # detr_whole_img, detr_in_bbox, detr_out_bbox = eval_metric_percent_above(detr_grayscale_cam, new_bbox_detr, threshold)
    detr_whole_img = eval_metric_percent_above_simple( detr_grayscale_cam, threshold )

    output_data.append( [image_names[i], threshold, yolo_predicted_label, yolo_predicted_label_prob, yolo_whole_img, detr_predicted_label, detr_predicted_label_prob, detr_whole_img] )

    # Delete garbage for ram
    gc.collect()

person_backpack.jpg
<class 'tuple'>
The max logit value is [tensor(7.3709, grad_fn=<AddBackward0>)]


/root/.cache/torch/hub/facebookresearch_detr_main/models/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


<class 'dict'>
The max logit value is [tensor(9.9013, grad_fn=<AddBackward0>), tensor(6.8809, grad_fn=<AddBackward0>)]
person-no_backpack.jpg
<class 'tuple'>
The max logit value is [tensor(8.1430, grad_fn=<AddBackward0>)]
<class 'dict'>
The max logit value is [tensor(6.6785, grad_fn=<AddBackward0>), tensor(9.9265, grad_fn=<AddBackward0>)]
person_car.jpeg
<class 'tuple'>
The max logit value is [tensor(14.8408, grad_fn=<AddBackward0>)]
<class 'dict'>
The max logit value is [tensor(13.1508, grad_fn=<AddBackward0>)]
person-no_car.jpeg
<class 'tuple'>
The max logit value is [tensor(6.3184, grad_fn=<AddBackward0>)]
<class 'dict'>
The max logit value is [tensor(13.4615, grad_fn=<AddBackward0>)]
person_backpack.jpg
<class 'tuple'>
The max logit value is [tensor(7.3709, grad_fn=<AddBackward0>)]
<class 'dict'>
The max logit value is [tensor(9.9013, grad_fn=<AddBackward0>), tensor(6.8809, grad_fn=<AddBackward0>)]
person-no_backpack.jpg
<class 'tuple'>
The max logit value is [tensor(8.1430, grad_f

In [ ]:
df = pd.DataFrame (output_data, columns=headers)
df.to_csv(output_file)

In [ ]:
output_images_dir = proj_dir + '/Q7/trial_2/'
images_to_visualize = list( range( len( image_names ) ) )
threshold = 0.7
generate_vis_analysis = True

In [ ]:
if(generate_vis_analysis):
  for i in images_to_visualize:
    # Create input
    test_img = Image.open( dataset_folder + '/' + image_names[i] ).convert('RGB')
    yolo_path = output_images_dir + image_names[i] + '_yolo_whole'
    yolo_path_t = output_images_dir + image_names[i] + '_yolo_thres'
    detr_path = output_images_dir + image_names[i] + '_detr_whole'
    detr_path_t = output_images_dir + image_names[i] + '_detr_thres'

    # orig_bbox = all_boxes[i]
    # orig_img_shape = np.array(test_img).shape

    # Create target lable
    target_name = image_names[i].split( '-' )[0]
    target_name = target_name.split( '_' )[0]

    #
    # YOLO
    #
    
    # YOLO Target label
    # handle mismatch spelling in yolo classes
    yolo_target_name = target_name
    if yolo_target_name == 'airplane':
      yolo_target_name = 'aeroplane'
    if yolo_target_name == 'dining table':
      yolo_target_name = 'diningtable'
      
    target_category = [index for index, class_instance in enumerate(yolo_classes) if class_instance == yolo_target_name]

    # Input
    input_tensor = yolo_transform(test_img)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # Grad cam time
    yolo_cam = GradCAM(model=yolo_model, target_layers=yolo_target_layers, use_cuda=False, reshape_transform=None, classification_logit="pred_logits")
    yolo_grayscale_cam = yolo_cam(input_tensor=input_batch, target_category=target_category, eigen_smooth=False, aug_smooth=False)
    yolo_grayscale_cam = yolo_grayscale_cam[0, :]

    # process image to write gradcam on
    rgb_image = np.array(yolo_transform_eval(test_img)).astype(np.float32)
    rgb_image = rgb_image/255

    # whole image gradcam visualization
    yolo_heatmap, yolo_visualization = show_cam_on_image_updated(rgb_image, yolo_grayscale_cam, use_rgb=False)
    cv2.imwrite(yolo_path, yolo_visualization)

    # resize bbox and calculate metrics
    #new_bbox_yolo = resizeBbox(orig_bbox, (orig_img_shape[0], orig_img_shape[1]), (yolo_input_image_shape[0], yolo_input_image_shape[1]))
    #yolo_visualized_cam_threshold = visualize_eval_metric(rgb_image, yolo_grayscale_cam, yolo_heatmap, new_bbox_yolo, threshold)
    #cv2.imwrite(yolo_path_t, yolo_visualized_cam_threshold)


    ######


    #
    # DETR
    #

    # DETR Target Label
    target_category = [index for index, class_instance in enumerate(DETR_CLASSES) if class_instance == target_name]

    # Input
    input_tensor = detr_transform(test_img)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # Grad cam time
    detr_cam = GradCAM(model=detr_model, target_layers=detr_target_layers, use_cuda=False, reshape_transform=None, classification_logit="pred_logits")
    detr_grayscale_cam = detr_cam(input_tensor=input_batch, target_category=target_category, eigen_smooth=False, aug_smooth=False)
    detr_grayscale_cam = detr_grayscale_cam[0, :]

    # process image to write gradcam on
    rgb_image = np.array(detr_transform_eval(test_img)).astype(np.float32)
    rgb_image = rgb_image/255

    # whole image gradcam visualization
    detr_heatmap, detr_visualization = show_cam_on_image_updated(rgb_image, detr_grayscale_cam, use_rgb=False)
    cv2.imwrite(detr_path, detr_visualization)

    # resize bbox and calculate metrics
    #detr_input_image_shape = (detr_grayscale_cam.shape[0], detr_grayscale_cam.shape[1], 3)
    #new_bbox_detr = resizeBbox(orig_bbox, (orig_img_shape[0], orig_img_shape[1]), (detr_input_image_shape[0], detr_input_image_shape[1]))
    #detr_visualized_cam_threshold = visualize_eval_metric(rgb_image, detr_grayscale_cam, detr_heatmap, new_bbox_detr, threshold)
    #cv2.imwrite(detr_path_t, detr_visualized_cam_threshold)


<class 'tuple'>
The max logit value is [tensor(7.3709, grad_fn=<AddBackward0>)]


/root/.cache/torch/hub/facebookresearch_detr_main/models/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


<class 'dict'>
The max logit value is [tensor(9.9013, grad_fn=<AddBackward0>), tensor(6.8809, grad_fn=<AddBackward0>)]
<class 'tuple'>
The max logit value is [tensor(8.1430, grad_fn=<AddBackward0>)]
<class 'dict'>
The max logit value is [tensor(6.6785, grad_fn=<AddBackward0>), tensor(9.9265, grad_fn=<AddBackward0>)]
<class 'tuple'>
The max logit value is [tensor(14.8408, grad_fn=<AddBackward0>)]
<class 'dict'>
The max logit value is [tensor(13.1508, grad_fn=<AddBackward0>)]
<class 'tuple'>
The max logit value is [tensor(6.3184, grad_fn=<AddBackward0>)]
<class 'dict'>
The max logit value is [tensor(13.4615, grad_fn=<AddBackward0>)]
